# Notebook for SDD Hackathon 2024 wt Vortex.io 

## Library imports and dowloads

In [ ]:
!git clone "https://github.com/facebookresearch/segment-anything.git"
!pip install -q supervision --upgrade supervision

In [ ]:
import cv2
import supervision as sv
import torch
import numpy as np
import matplotlib.pyplot as plt
import os
from tqdm import tqdm
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
from sklearn.metrics.pairwise import cosine_similarity
import pickle
import pandas as pd

## SAM initianlization

In [ ]:
%cd segment-anything

In [ ]:
!wget -q https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth

In [ ]:
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor

DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
MODEL_TYPE = "vit_h"


CHECKPOINT_PATH = "./sam_vit_h_4b8939.pth"
sam = sam_model_registry[MODEL_TYPE](checkpoint=CHECKPOINT_PATH).to(device=DEVICE)
mask_generator = SamAutomaticMaskGenerator(sam)

## Training

### Fragmentation

In [ ]:
def get_fragments(image_path):
    image_bgr = cv2.imread(image_path)
    image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)
    sam_result = mask_generator.generate(image_rgb)
    masks = [o["segmentation"] for o in sam_result]
    all_mask = np.zeros(image_rgb.shape[:2])
    for mask in masks:
        all_mask = np.logical_or(all_mask, mask)
    remaining = np.logical_not(all_mask)
    fragments = [cv2.resize(remaining.astype('float32'), (320, 240)).astype(int)]
    fragments.extend([cv2.resize(mask.astype('float32'), (320, 240)).astype(int) for mask in masks])
    return fragments

def aggregate_fragment(fragments):
    mask = np.zeros((240,320))
    for i, frag in enumerate(fragments):
        h, w = frag.shape
        if h != 240 and w != 320:
            frag = cv2.resize(frag.astype("float32"), (320, 240)).astype(int)
        if np.max(frag) > 1:
            print("Max is not 1")
        mask = mask + frag * (i+1)
    return mask.astype("uint8")

In [ ]:
train_images_path = "/kaggle/input/water-segmentation-vtx/dataset/trainset/images/"
train_masks_path = "/kaggle/input/water-segmentation-vtx/dataset/trainset/masks/"
test_images_path = "/kaggle/input/water-segmentation-vtx/dataset/testset/images/"
files = os.listdir(train_images_path)
all_fragments = {}
cluster_fragments = {}
# num_choices = 100
num_choices = len(files)

count = 1
cluster = 1
for file in tqdm(np.random.choice(files, size=num_choices)):
    if count%150 == 0:
        share_train_frags = {}
        for file in cluster_fragments:
            share_train_frags[file] = aggregate_fragment(cluster_fragments[file])
        with open(f'../training_fragments_{cluster}.pkl', 'wb') as f:
            pickle.dump(share_train_frags, f)
        cluster += 1
        all_fragments = {**all_fragments, **cluster_fragments}
        cluster_fragments = {}
    cluster_fragments[file] = get_fragments(train_images_path + file)
    count+=1

 52%|█████▏    | 461/891 [43:04<39:52,  5.56s/it]  

### Fragment labelization

In [ ]:
def get_label(frag, file, show=False):
    mask_file = train_masks_path + file.replace(
        "jpg", "png"
    )
    mask = cv2.imread(mask_file)
    mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)//255
    test = mask * frag
    if show:
        plt.subplot(1,3,1)
        img = cv2.imread(train_images_path + file)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        plt.imshow(img)
        plt.subplot(1,3,2)
        plt.imshow(mask, vmin=0, vmax=1)
        plt.subplot(1,3,3)
        plt.imshow(frag, vmin=0, vmax=1)
        plt.show()
    if np.sum(test) == np.sum(frag):
        return True
    return np.sum(test) / np.sum(frag) > .7


### Fragment encoding

In [ ]:
# Load a pretrained ResNet model
model = models.resnet50(weights="ResNet50_Weights.DEFAULT")
# Remove the classification layer
model = torch.nn.Sequential(*list(model.children())[:-1])
# Set the model to evaluation mode
model.eval()

# Define preprocessing transformations
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

def get_features(frag):
    input_tensor = preprocess(Image.fromarray(frag))
    input_batch = input_tensor.unsqueeze(0)
    with torch.no_grad():
        features = model(input_batch)
    return features.view(-1).numpy()

### Fragment aggregation

In [ ]:
def add_file_fragments(directory, file, fragments, training_set):
    origin = cv2.imread(directory+file)
    origin = cv2.cvtColor(origin, cv2.COLOR_BGR2RGB)
    for frag in fragments:
        h, w, _ = origin.shape
        resized_frag = cv2.resize(frag.astype('float32') , (w,h))
        fragment = np.zeros(origin.shape)
        for i in range(3):
            fragment[:,:,i] = origin[:,:,i] * resized_frag.astype(int)
        training_set.append(
            {
                "filename": file,
                "label": get_label(resized_frag, file),
                "features": get_features(fragment.astype("uint8"))
            }
        )
    

# with open('training_fragments.pkl', 'rb') as f:
#     all_fragments = pickle.load(f)
training_set = []
for file in tqdm(all_fragments):
    fragments = all_fragments[file]
    add_file_fragments(train_images_path, file, fragments, training_set)
print(f"{len(training_set)} fragments in training set.")
print()
# print("Saving fragments...")
# with open('training_set.pkl', 'wb') as f:
#     pickle.dump(training_set, f)

## Testing

### Fragmentation

In [ ]:
test_fragments = {}
print("Fragmenting pictures...")
files = os.listdir(test_images_path)
for file in tqdm(files):
    test_fragments[file] = get_fragments(test_images_path + file)
print("Done!")
with open(f'../testing_fragments.pkl', 'wb') as f:
    pickle.dump(test_fragments, f)

### Encoding

In [ ]:
print("Getting fragment's features...")
testing_set = []
for file in tqdm(test_fragments):
    fragments = test_fragments[file]
    origin = cv2.imread(test_images_path+file)
    origin = cv2.cvtColor(origin, cv2.COLOR_BGR2RGB)
    h, w, _ = origin.shape
    for frag in fragments:
        fragment = np.zeros(origin.shape)
        for i in range(3):
            fragment[:,:,i] = origin[:,:,i] * cv2.resize(frag.astype("float32"), (w, h)).astype(int)
        testing_set.append(
            {
                "filename": file,
                "mask": frag.astype(int),
                "features": get_features(fragment.astype("uint8"))
            }
        )
print("Done!")
print()
# print("Saving fragments...")
# with open('testing_set.pkl', 'wb') as f:
#     pickle.dump(testing_set, f)

### Labelization

In [ ]:
def is_frag_water(frag, show=False):
    scores = [cosine_similarity(frag["features"].reshape(1,-1), 
                               o["features"].reshape(1,-1)
                              )[0][0] 
              for o in training_set]
    best = np.argmax(scores)
    if show:
        plt.imshow(training_set[best]["fragment"])
        plt.show()
    return training_set[best]["label"]


print("Labeling test fragments...")
# with open('testing_set.pkl', 'rb') as f:
#     testing_set = pickle.load(f)
test_features = [o['features'] for o in testing_set]
train_features = [o['features'] for o in training_set]
similarity = cosine_similarity(test_features, train_features)
for i, frag in tqdm(enumerate(testing_set)):
    frag["label"] = training_set[np.argmax(similarity[i])]["label"]
print("Done!")

### Mask creation 

In [ ]:
print("Merging water masks...")
testing_masks = {}
for frag in tqdm(testing_set):
    file = frag["filename"]
    if frag["label"]:
        if file in testing_masks:
            mask = frag["mask"]
            testing_masks[file]["mask"] = testing_masks[file]["mask"] + cv2.resize(mask.astype("float32"), testing_masks[file]["shape"])
        else:
            mask = {}
            h, w, _ = cv2.imread(test_images_path+file).shape
            mask["shape"] = (w, h)
            mask["mask"] = cv2.resize(frag["mask"].astype("float32"), mask["shape"])
            testing_masks[file] = mask
print("Done!")

### Submission file generation

In [ ]:
def boolean_array_to_rle(boolean_array):
    boolean_vector = boolean_array.copy().reshape(1,-1)[0]
    rle = []
    current_idx = 0
    current_value = -1
    for i in range(len(boolean_vector)):
        if boolean_vector[i] >= 1:
            if current_value != 1:
                current_value = 1
                current_idx = i+1
        else:
            if current_value >= 1:
                current_value = 0
                rle.append((current_idx, i-current_idx+1))
                current_idx = i+1
    return rle

def rle_to_str(rle):
    return " ".join([f"{e[0]} {e[1]}" for e in rle])

print("Creating submission file...")
files = []
rles = []
for file in tqdm(testing_masks):
    mask = testing_masks[file]["mask"]
    name = file.replace("jpg", "png")
    rle = rle_to_str(boolean_array_to_rle(mask))
    files.append(name)
    rles.append(rle)
df = pd.DataFrame({'img_key': files, 'rle_mask': rles})
df.to_csv("../submission.csv", index=False)
print("Done!")

### Visualisation

In [ ]:
visu_files = np.random.choice(files, 10)
for i, file in enumerate(visu_files):
    plt.figure(figsize=(12,6))
    plt.subplot(1,2,1)
    origin = cv2.imread(test_images_path+file.replace("png","jpg"))
    origin = cv2.cvtColor(origin, cv2.COLOR_BGR2RGB)
    plt.imshow(origin)
    plt.imshow(testing_masks[file.replace("png","jpg")]['mask'], alpha=.2, cmap='jet', vmax=1, vmin=0)
    plt.subplot(1,2,2)
    plt.imshow(testing_masks[file.replace("png","jpg")]['mask'], cmap='jet', vmin=0, vmax=1)
    plt.show()

In [ ]:
share_test_set = {}
for file in tqdm(test_fragments):
    fragments = test_fragments[file]
    share_test_set[file] = aggregate_fragment(fragments)
    
print(list(share_test_set.items())[0])

import sys
sys.getsizeof(share_test_set)

In [ ]:
with open('../share_test_set.pkl', 'wb') as f:
    pickle.dump(share_test_set, f)